In [1]:
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet
from automates.gromet.query import query
import gpt3 as gpt
import requests

The following assumes that you have already generated the json (e.g., by running the notebook script `py_src_to_grometFN_JSON.ipynb`)

In [2]:
gromet_fn_module = json_to_gromet("gromet/CHIME_SIR_while_loop--Gromet-FN-auto.json")

The following uses the simple query to collect all named output ports of a GrometFNModule.
`nops` will hold a `List` of `Tuples`, where each tuple has the following format:
```
(<name_of_output_port>,    # named output port == a variable in source code
 <literal_value>,          # IF a literal value has been assigned, otherwise None
 <source_code_reference>)  # metadatum for the source code location of the assignment
```

In [8]:
nops = query.collect_named_output_ports(gromet_fn_module)
print(nops[0])

('inv_contact_rate', None, {'code_file_reference_uid': '09a64930-e9e4-8ae5-f825-bdf7c1e729f9',
 'col_begin': 4,
 'col_end': 50,
 'line_begin': 3,
 'line_end': 3,
 'metadata_type': 'source_code_reference',
 'provenance': {'method': 'skema_code2fn_program_analysis',
                'timestamp': '2022-11-30 16:18:33.012848'}})


We first set up access to the MIRA Epidemiology Domain Knowledge Graph (DKG) web service.

In [4]:
MIRA_DKG_URL = 'http://34.230.33.149:8771'

def get_mira_dkg_term(term, attribs):
    res = requests.get(MIRA_DKG_URL + '/api/search', params={'q': term})
    term = [entity for entity in res.json() if entity['id'].startswith('askemo')][0]
    res = {attrib: term.get(attrib) for attrib in attribs if term.get(attrib) is not None}
    return res

Next, we choose a set of terms and attributes to pull into a local ontology that we can use to connect to the model. In addition, for some of the terms, we define custom ranges in which their values for this model can be adjusted.

In [5]:
# Terms we want to find in MIRA and specific attributes we want to add to our local ontology
terms = ['population', 'doubling time', 'recovery time', 'infectious time']
attribs = ['description', 'synonyms', 'xrefs', 'suggested_unit', 'suggested_data_type',
           'physical_min', 'physical_max', 'typical_min', 'typical_max']

# The local ontology if filled up from the MIRA DKG
LOCAL_ONTOLOGY = {
    term: get_mira_dkg_term(term, attribs) for term in terms
    }

# We can also set further local / use-case specific constraints as needed
LOCAL_ONTOLOGY['population']['typical_min'] = 1000
LOCAL_ONTOLOGY['population']['typical_max'] = 40_000_000

LOCAL_ONTOLOGY

{'population': {'description': 'The number of people who live in an area being modeled.',
  'synonyms': [],
  'xrefs': [{'id': 'ido:0000509', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'person',
  'suggested_data_type': 'int',
  'physical_min': 0.0,
  'typical_min': 1000,
  'typical_max': 40000000},
 'doubling time': {'description': 'The length of time that an infectious disease requires to double in incidence.',
  'synonyms': [{'value': 'doubling rate',
    'type': 'oboInOwl:hasRelatedSynonym'}],
  'xrefs': [{'id': 'cemo:epidemic_doubling_time', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'recovery time': {'description': 'The length of time an infected individual needs to recover after being infected.',
  'synonyms': [{'value': 'mean recovery time',
    'type': 'oboInOwl:hasExactSynonym'}],
  'xrefs': [],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'infectious time': 

Ontology to Gromet matching

In [ ]:
targets = ['population', 'infectious time']
terms = ['population', 'doubling time', 'recovery time', 'infectious time']

parameters = set()
var_dict = {}
for nop in nops:
    if nop[1] is not None:
        parameters.add(nop[0])
        var_dict[nop[0]] = nop

discoveredParameterConnections = gpt.match_gromet_targets(targets, list(parameters), var_dict, terms)
discoveredParameterConnections

Ontology to code matching

In [ ]:
code = "model/SIR/CHIME_SIR_while_loop.py"
targets = ['population', 'infectious time']
discoveredParameterConnections = []
try:
    discoveredParameterConnections = gpt.match_code_targets(targets, code, terms)
except OpenAIError as err:
    print("OpenAI connection error:", err)
    print("Using hard-coded connections")
    discoveredParameterConnections = [("infectious time", {"name": "grometSubObject"}, 14.0, 67),("population", {"name": "grometSubObject"}, 1000, 80)]

discoveredParameterConnections

Formula to code matching

In [ ]:
gpt.formula_code_connection()

Code to text matching

In [ ]:
gpt.code_text_connection()

In [ ]:
# WIP 
from ipywidgets import interact, widgets
from IPython.display import display

text = widgets.Text(
    value='last',
    placeholder='Paste ticket description here!',
    description='String:',
    disabled=False
)
display(text)

def callback(wdgt):
    # replace by something useful
    display(wdgt.value)

text.on_submit(callback)

In [ ]:
# WIP
def mytextfunction(x):
    print(x)
    return x

interactive_text = interactive(mytextfunction, x="Hello World")
interactive_text

In [1]:
# WIP 
from ipywidgets import interact, widgets
from IPython.display import display

text = widgets.Text(
    value='last',
    placeholder='Paste ticket description here!',
    description='String:',
    disabled=False
)
display(text)

def callback(wdgt):
    # replace by something useful
    display(wdgt.value)

text.on_submit(callback)

Text(value='last', description='String:', placeholder='Paste ticket description here!')

In [2]:
# WIP
def mytextfunction(x):
    print(x)
    return x

interactive_text = interactive(mytextfunction, x="Hello World")
interactive_text

NameError: name 'interactive' is not defined